In [14]:
from datasets import load_dataset

dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"]
validation_set = dataset["validation"]

# Define the languages of interest
languages = ["arabic", "bengali", "indonesian"]
validation_set

Dataset({
    features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
    num_rows: 13325
})

In [15]:

# unanswerable = train_set[train_set["annotations"]["answer_start"][0] == -1]
unanswerable = train_set.filter(lambda example: example["annotations"]["answer_start"][0] == -1)
answerable = train_set.filter(lambda example: example["annotations"]["answer_start"][0] != -1)
unanswerable[0]

{'question_text': 'Milloin Charles Fort syntyi?',
 'document_title': 'Charles Fort',
 'language': 'finnish',
 'annotations': {'answer_start': [-1], 'answer_text': ['']},
 'document_plaintext': 'Fortin ystävät perustivat Fortean Societyn jo hänen elinaikanan, ja seuraa johti kirjailija Tiffany Thayer, puoliksi tosissaan ja puoliksi vitsin vuoksi, kuten Fortin itsensä työ. Fort kuitenkin torjui Societyn ja ankarana autoritaarisuuden vastustajana kieltäytyi sen johtajuudesta ja edelleen vastusti seuraa, sillä se saattaisi houkutella spiritualisteja, kiihkoilijoita ja niitä, jotka vastustivat tiedettä, koska se ei ollut hyväksynyt heitä. Se houkuttelisi niitä, jotka uskoivat omaan lempi-ilmiöönsä ylitse muiden: täysin vastakohtainen asenne fortilaisuudelle. On siis ironista, että sittemmin useita sellaisia "fortilaisia" seuroja on perustettu lisää. ',
 'document_url': 'https://fi.wikipedia.org/wiki/Charles%20Fort'}

In [16]:
def oracle(answer: str, answer_start: int, document: str) -> bool:
    """Returns True if the answer is in the document at the given index."""
    return answer_start != -1 and document[answer_start:answer_start + len(answer)] == answer

In [17]:
def classifier_1(question: str, document: str, language: str) -> bool:
    return True # question in document

In [18]:
def classifier_bow(question: str, document: str, language: str) -> bool:
    """Returns True if the question is in the document."""
    tracker = [word in document for word in question.split()]
    contained_ratio = sum(tracker) / len(tracker)
    return contained_ratio > 0.4

In [19]:
from bpemb import BPEmb

language_to_bpe = {'bengali': BPEmb(lang="bn", dim=50), 'indonesian': BPEmb(lang="id", dim=50), 'arabic': BPEmb(lang="ar", dim=50)}

def classifier_bpe(question: str, document: str, language: str) -> bool:
    bpemb_model = language_to_bpe[language]
    # Tokenize question and document
    question_bpe = bpemb_model.encode(question)
    document_bpe = bpemb_model.encode(document)
    tracker = [token_bpe in document_bpe for token_bpe in question_bpe]
    contained_ratio = sum(tracker) / len(tracker)
    return contained_ratio > 0.2

In [20]:
import pandas as pd

# Try out the classifiers and evaluate them using the oracle function
results = []
for classifier in [classifier_1, classifier_bow, classifier_bpe]:
    for sample in validation_set:
        result = {}
        if sample['language'] not in languages:
            continue
        
        result['classifier'] = classifier.__name__
        result['language'] = sample['language']
        
        classification = classifier(sample["question_text"], sample["document_plaintext"], sample['language'])
        truth = oracle(sample["annotations"]["answer_text"][0], sample["annotations"]["answer_start"][0], sample["document_plaintext"])
        result['correct'] = classification == truth
        
        results.append(result)
        
results = pd.DataFrame(results)


In [21]:
# Display results
for language in languages:
    lang_results = results[results['language'] == language]
    print(f"Results for {language}")
    
    # Calculate the mean correctness for each classifier
    classifier_mean = lang_results.groupby('classifier')['correct'].mean().reset_index()
    print( classifier_mean)
    
    # Calculate the number of rows (samples) for each classifier
    classifier_count = len(lang_results)
    print("Count:", classifier_count)
    print()


Results for arabic
       classifier   correct
0    classifier_1  0.500000
1  classifier_bow  0.707676
2  classifier_bpe  0.691903
Count: 5706

Results for bengali
       classifier   correct
0    classifier_1  0.500000
1  classifier_bow  0.714286
2  classifier_bpe  0.723214
Count: 672

Results for indonesian
       classifier   correct
0    classifier_1  0.501259
1  classifier_bow  0.669186
2  classifier_bpe  0.690176
Count: 3573

